In [25]:
import pandas as pd 
import requests 
import json 
from urllib.parse import quote, unquote
import os
import configparser

# google bigquery:
from google.cloud import bigquery 
from google.oauth2 import service_account

In [26]:
configfile = configparser.ConfigParser()
configfile.read("config.cfg")
print(configfile.sections())

# set SSL cert vals, not necessary unless there's SSL cert error
os.environ["REQUESTS_CA_BUNDLE"] = configfile['SSL']['REQUESTS_CA_BUNDLE']
os.environ["SSL_CERT_FILE"] = configfile['SSL']['SSL_CERT_FILE']

# google IAM credentials
#gclient = bigquery.Client()
credentials = service_account.Credentials.from_service_account_file(configfile['GOOGLE']['ACCESS_KEY'])

# Initialize BigQuery client
gclient = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id
)

# # List datasets in the project
# datasets = list(gclient.list_datasets())

# if datasets:
#     print("Datasets in project:")
#     for dataset in datasets:
#         print(f"- {dataset.dataset_id}")

['SSL', 'GOOGLE']


In [ ]:
project_id = configfile['GOOGLE']['PROJECT_ID']
client = bigquery.Client(credentials=credentials, project=project_id)

# Define dataset ID
dataset_id = f"crash_data"

# Create dataset on BigQuery
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Dataset {dataset_id} created.")

# Define the schema for the collision data table
collision_data_schema = [
    bigquery.SchemaField("collision_id", "INTEGER", mode="NULLABLE"),
    # bigquery.SchemaField("crash_date", "DATE"),
    # bigquery.SchemaField("crash_time", "TIME"),
    # creating a datetime filed instead of separate date and time fields for better query performance
    bigquery.SchemaField("crash_datetime", "DATETIME")
    bigquery.SchemaField("borough", "STRING"),
    bigquery.SchemaField("zip_code", "STRING"),
    bigquery.SchemaField("latitude", "FLOAT"),
    bigquery.SchemaField("longitude", "FLOAT"),
    bigquery.SchemaField("location", "GEOGRAPHY"),
    bigquery.SchemaField("on_street_name", "STRING"),
    bigquery.SchemaField("cross_street_name", "STRING"),
    bigquery.SchemaField("off_street_name", "STRING"),
    bigquery.SchemaField("number_of_persons_injured", "INTEGER"),
    bigquery.SchemaField("number_of_persons_killed", "INTEGER"),
    bigquery.SchemaField("number_of_pedestrians_injured", "INTEGER"),
    bigquery.SchemaField("number_of_pedestrians_killed", "INTEGER"),
    bigquery.SchemaField("number_of_cyclist_injured", "INTEGER"),
    bigquery.SchemaField("number_of_cyclist_killed", "INTEGER"),
    bigquery.SchemaField("number_of_motorist_injured", "INTEGER"),
    bigquery.SchemaField("number_of_motorist_killed", "INTEGER"),
    bigquery.SchemaField("contributing_factor_vehicle_1", "STRING"),
    bigquery.SchemaField("contributing_factor_vehicle_2", "STRING"),
    bigquery.SchemaField("vehicle_type_code1", "STRING"),
    bigquery.SchemaField("vehicle_type_code2", "STRING"),
]

# Create the table reference and create the table
collision_table = bigquery.Table(dataset_ref.table("collision_data"), schema=collision_data_schema)
client.create_table(collision_table, exists_ok=True)

# Specify the destination table reference (without the project ID prefix)
table_ref = client.dataset(dataset_id).table("collision_data")

# Configure the job options (you don't need to specify CSV format as it's inferred)
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")


In [ ]:
## get data from open data 
# ignoring the location column here but will add in the dataframe as Point(lat,lon)
base_url = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?"
api_query = """SELECT 
                    collision_id,
                    crash_date,
                    crash_time,
                    borough,
                    zip_code, 
                    latitude,
                    longitude,
                    
                    on_street_name,
                    cross_street_name,
                    off_street_name, 
                    number_of_persons_injured,
                    number_of_persons_killed,
                    number_of_pedestrians_injured,
                    number_of_pedestrians_killed,
                    number_of_cyclist_injured,
                    number_of_cyclist_killed,
                    number_of_motorist_injured,
                    number_of_motorist_killed,
                    contributing_factor_vehicle_1,
                    contributing_factor_vehicle_2,
                    vehicle_type_code1,
                    vehicle_type_code2
                LIMIT 10000
                
            """

In [37]:
url = base_url + "$query=" + api_query
resp = requests.get(url)

In [38]:
df_opendata = pd.DataFrame(resp.json())

In [39]:
df_opendata[['crash_date', 'crash_time', 'latitude', 'longitude']]

,crash_date,crash_time,latitude,longitude
0,2021-09-11T00:00:00.000,2:39,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,NaN,NaN
2,2023-11-01T00:00:00.000,1:29,40.62179,-73.970024
3,2022-06-29T00:00:00.000,6:55,NaN,NaN
4,2022-09-21T00:00:00.000,13:21,NaN,NaN
...,...,...,...,...
9995,2021-09-18T00:00:00.000,10:50,NaN,NaN
9996,2021-09-18T00:00:00.000,1:00,40.64315,-73.97548
9997,2021-09-17T00:00:00.000,9:10,40.713314,-73.78522
9998,2021-09-18T00:00:00.000,4:48,40.605244,-73.97233


In [ ]:
# location column in Open Data is not very helpful by itself. will just combine lat lon columns to keep things simple
df_opendata['crash_date'] = pd.to_datetime(df_opendata['crash_date']).dt.date 
df_opendata['crash_time'] = pd.to_datetime(df_opendata['crash_time'], format="mixed").dt.time
df_opendata['latitude'] = df_opendata['latitude'].astype('float')
df_opendata['longitude'] = df_opendata['latitude'].astype('float')
df_opendata['collision_id'] = df_opendata['collision_id'].astype('int')
df_opendata['number_of_persons_injured'] = df_opendata['number_of_persons_injured'].astype('int')
df_opendata['number_of_persons_killed'] = df_opendata['number_of_persons_killed'].astype('int')
df_opendata['number_of_pedestrians_injured'] = df_opendata['number_of_pedestrians_injured'].astype('int')
df_opendata['number_of_pedestrians_killed'] = df_opendata['number_of_pedestrians_killed'].astype('int')
df_opendata['number_of_cyclist_injured'] = df_opendata['number_of_cyclist_injured'].astype('int')
df_opendata['number_of_cyclist_killed'] = df_opendata['number_of_cyclist_killed'].astype('int')
df_opendata['number_of_motorist_injured'] = df_opendata['number_of_motorist_injured'].astype('int')
df_opendata['number_of_motorist_killed'] = df_opendata['number_of_motorist_killed'].astype('int')
df_opendata['location'] = df_opendata.apply(lambda row: f"Point({row['longitude']} {row['latitude']})" if pd.notnull(row["latitude"]) and pd.notnull(row["longitude"]) else None, axis=1)
# create crash_datetime column by combining the date and time columns
# Combine into a single datetime column
# df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
df_opendata['crash_datetime'] = pd.to_datetime(df_opendata['crash_date'].astype(str) + ' ' + df_opendata['crash_time'].astype(str))


In [47]:
df_opendata[['crash_date', 'crash_time', 'crash_datetime','latitude', 'longitude', 'location']]

,crash_date,crash_time,crash_datetime,latitude,longitude,location
0,2021-09-11,02:39:00,2021-09-11 02:39:00,NaN,NaN,None
1,2022-03-26,11:45:00,2022-03-26 11:45:00,NaN,NaN,None
2,2023-11-01,01:29:00,2023-11-01 01:29:00,40.621790,40.621790,Point(40.62179 40.62179)
3,2022-06-29,06:55:00,2022-06-29 06:55:00,NaN,NaN,None
4,2022-09-21,13:21:00,2022-09-21 13:21:00,NaN,NaN,None
...,...,...,...,...,...,...
9995,2021-09-18,10:50:00,2021-09-18 10:50:00,NaN,NaN,None
9996,2021-09-18,01:00:00,2021-09-18 01:00:00,40.643150,40.643150,Point(40.64315 40.64315)
9997,2021-09-17,09:10:00,2021-09-17 09:10:00,40.713314,40.713314,Point(40.713314 40.713314)
9998,2021-09-18,04:48:00,2021-09-18 04:48:00,40.605244,40.605244,Point(40.605244 40.605244)


In [49]:
# making sure to filter the only columns they are valid for bq dataset
# df_opendata.columns
bq_columns = [
    'collision_id', 
    # 'crash_date', 
    # 'crash_time', 
    'crash_datetime'
    'on_street_name',
    'off_street_name', 
    'number_of_persons_injured',
    'number_of_persons_killed', 
    'number_of_pedestrians_injured',
    'number_of_pedestrians_killed', 
    'number_of_cyclist_injured',
    'number_of_cyclist_killed', 
    'number_of_motorist_injured',
    'number_of_motorist_killed', 
    'contributing_factor_vehicle_1',
    'contributing_factor_vehicle_2', 
    'vehicle_type_code1',
    'vehicle_type_code2', 
    'borough', 'zip_code', 
    'latitude', 
    'longitude',
    'cross_street_name', 
    'location']

## upload dataframe to bigquery dataset

In [ ]:

# Load data into BigQuery table from DataFrame
job = client.load_table_from_dataframe(df_opendata[bq_columns], table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print("Data uploaded successfully!")


Dataset crash_data created.
Data uploaded successfully!


In [24]:
df_opendata[['number_of_persons_injured', 'number_of_persons_killed']]

,number_of_persons_injured,number_of_persons_killed
0,2,0
1,1,0
2,1,0
3,0,0
4,0,0
...,...,...
9995,2,0
9996,0,0
9997,0,0
9998,0,0
